In [2]:
import numpy as np
import matplotlib.pyplot as plt
import random
import math
from copy import copy, deepcopy
%matplotlib inline




In [3]:
nb_gen = 1000
nb_pop = 100

elites_percentage = 0.1
num_elites = math.floor(nb_pop*elites_percentage)

a = 10
b1 = 10
b2 = 10
c = 10

mutation_rate = 1/(7*73*a*b1*b2*c)

In [4]:
lignes = [i for i in range(0,8)]
columns = [i for i in range(0,7)]
nb_lignes = 6
nb_colonnes = 7

In [5]:
while(1):
    if(1):
        break;

In [6]:
def init_grid():
    return (np.zeros((8,9)))

In [7]:
def count_placed_column(grid,column):
    a = 0
    for ligne in range(1,7):
        a+= abs(grid[ligne][column])
    return int(a)

In [8]:
def make_move(grid, column,side):
    ligne = count_placed_column(grid,column)
    if ligne >= 6:
        return [[1]]
    else:
        grid[ligne+1][column] = side
        return grid
        


In [9]:
def nb_pions_dir(grid, i, j, inDirX, inDirY) :
    side = copy(grid[i][j])
    lig,col = i,j
    nb_pion = copy(side)
    lig,col = lig+inDirX,col+inDirY
    while (grid[lig][col] == side):
        nb_pion+=side
        lig,col = lig+inDirX,col+inDirY
    lig,col  = i-inDirX,j-inDirY
    while (grid[lig][col] == side):
        nb_pion+=side
        lig,col = lig-inDirX,col-inDirY
    return nb_pion
        
    
        

In [10]:
def is_won(game,i,j):
    side = game[i][j]
    #print(side)
    #print("counting x axis")
    a = side*nb_pions_dir(game,i,j,1,0)
    #print("counting y axis")
    b = side*nb_pions_dir(game,i,j,0,1)
    #print("counting diag bas axis")
    c = side*nb_pions_dir(game,i,j,1,1)
    #print("counting diag haut axis")
    d = side*nb_pions_dir(game,i,j,1,-1)
    #print("le max est "+str(side*max(a,b,c,d)))
    #print(a,b,c,d)
    return(abs(max(a,b,c,d)) >= 4)

In [11]:
class Bot : 
    def __init__(self):
        self.A = np.random.random((a,73))
        self.B = np.random.random((b1,b2))
        self.C= np.random.random((c,7))
        self.fitness = 0

    def make_decision(self,grid):
        map_vect = np.reshape(grid,72)
        map_vect = np.insert(map_vect,0,self.side)
        output = self.A.dot(map_vect).dot(self.B).dot(self.C)
        i = np.argmax(output) +1
        #print(output)
        grid_copy = copy(grid)
        while (make_move(grid_copy,i,self.side)[0][0] == 1):
            i = random.randint(1,7)
        grid = make_move(grid, i,self.side)
        return i 
    
    def evaluate(self):
        self.fitness = self.run_neural_network()
        
    def mutate(self):
        child = Bot()
        child.A = copy(self.A)
        child.B = copy(self.B)
        child.C = copy(self.C)
        for i in child.A:
            if random.random() < mutation_rate:
                i = random.random()
        for i in child.B:
            if random.random() < mutation_rate:
                i = random.random()
        for i in child.C:
            if random.random() < mutation_rate:
                i = random.random()
        
        return child



In [23]:
def make_game(bot):
    grid = init_grid()
    print(grid)
    if random.random() > 0.5:
        bot.side = 1
        my_side = -1
        print("Vous jouez *")
    else:
        bot.side = -1
        my_side = 1
        print("Vous jouez -")
    
    if my_side == 1:
        while(1):
            print_grid(grid)
            my_decision = int(input("choisissez un move"))
            ligne = int(count_placed_column(grid,my_decision))
            grid = make_move(grid,my_decision,my_side)
            if is_won(grid,ligne+1,my_decision):
                print_grid(grid)
                print("game over")
                return
            
            decision1 = bot.make_decision(grid)
            ligne = int(count_placed_column(grid,decision1))
            if is_won(grid,ligne, decision1 ):
                print("game over")
                print_grid(grid)
                break
    else:
        while(1):

            decision1 = bot.make_decision(grid)
            ligne = int(count_placed_column(grid,decision1))
            if is_won(grid,ligne, decision1 ):
                print("game over")
                print_grid(grid)
                break
            print_grid(grid)
            my_decision = int(input("choisissez un move"))
            ligne = int(count_placed_column(grid,my_decision))
            grid = make_move(grid,my_decision,my_side)
            print_grid(grid)
            if is_won(grid,ligne+1,my_decision):
                print_grid(grid)
                print("game over")
                return
            
            
            

        #print(np.flipud(grid))
    
    
#bot1 = Bot()
#make_game(bot1)
    
    

In [13]:
def tournament_select(population):
    tournament = random.sample(population, 3)
    return sorted(tournament, key=lambda bot: bot.fitness, reverse = True)[0]

In [14]:
def uniform_crossover(p1,p2):
    child = copy(p1)
    
    
    for i in range(0,a):
        for j in range(0,73):
            if random.random() < 0.5:
                child.A[i,j] = p2.A[i,j]
    for i in range(0,b1):
        for j in range(0,b2):
            if random.random() < 0.5:
                child.B[i,j] = p2.A[i,j]
    for i in range(0,c):
        for j in range(0,7):
            if random.random() < 0.5:
                child.C[i,j] = p2.A[i,j]
    return child

In [15]:
def print_grid(grid):
    
    for i in range(8):
        ligne = []
        for j in range(9):
            if grid[i,j] == -1:
                ligne.append("*")
            elif grid[i,j] == 1:
                ligne.append("-")
            else :
                ligne.append("0")
        print(ligne)
    

In [16]:
def fight_bot(bot1,bot2):
    grid = init_grid()
    #print_grid(grid)
    nb_coups =0
    if(bot1.side == 1):
        while (1):
            decision1 = bot1.make_decision(grid)
            ligne = int(count_placed_column(grid,decision1))
            if is_won(grid,ligne, decision1 ):
                bot1.fitness += 1
                bot2.fitness += -1
                #print("winner : "+str(bot1.side))
                break
            #print_grid(grid)
            decision2 = bot2.make_decision(grid)
            ligne = int(count_placed_column(grid,decision2))
            if is_won(grid,ligne, decision2 ):
                bot1.fitness += -2
                bot2.fitness += 2
                #print("winner : "+str(bot2.side))
                #print(grid)
                break
            nb_coups +=2
            if nb_coups >40:
                bot1.fitness += 2
                bot1.fitness += 2
                #print_grid(grid)
                #print("it's a draw")
                break;
        else:
            while (1):
                decision2 = bot2.make_decision(grid)
                ligne = int(count_placed_column(grid,decision2))
                if is_won(grid,ligne, decision2 ):
                    bot2.fitness += 1
                    bot1.fitness += -1
                    #print("winner : "+str(bot2.side))
                    break
                #print_grid(grid)
                decision1 = bot1.make_decision(grid)
                ligne = int(count_placed_column(grid,decision1))
                if is_won(grid,ligne, decision1 ):
                    bot2.fitness += -2
                    bot1.fitness += 2
                    #print("winner : "+str(bot1.side))
                    #print(grid)
                    break
                nb_coups +=2
                if nb_coups >40:
                    bot2.fitness += 2
                    bot2.fitness += 2
                    #print_grid(grid)
                    #print("it's a draw")
                    break;
    #print_grid(grid)
    #print_grid(grid)
        
        

In [25]:
def evaluate_pop(population):
    fits = []
    for i in range(len(population)):
        population[i].fitness = 0
        for j in range(len(population)):
            if i != j:
                if random.random()>0.5:
                    population[i].side,population[j].side = 1,-1
                else:
                    population[i].side,population[j].side = -1,1   
                fight_bot(population[i],population[j])
        fits.append(population[i].fitness)
    return population


In [26]:
def list_fits(pop):
    l = []
    for i in pop:
        l.append(i.fitness)
    return(l)

In [30]:
select = tournament_select
crossover = uniform_crossover

def step_ga(population):
    population = evaluate_pop(population)
    population = sorted(population, key=lambda bot: bot.fitness, reverse = True)
    fits = list_fits(population)
    print(fits)
    
    new_population = []
    
    for i in range(nb_pop//4):
        new_population.append(population[i])
        new_population[i].mutate()
    
        
    while len(new_population) < len(population):
        parent1 = select(population)
        parent2 = select(population)
        child1 = crossover(parent1, parent2)
        #child1.evaluate()
        child1 = child1.mutate()
        #child1.evaluate()
        new_population.append(child1)
    return new_population




def ga(n_gen, mutation_rate):
    population = []
    
    for i in range(nb_pop):
        k = Bot()
        population.append(k)

    print("Starting generating")
    population = sorted(population, key=lambda bot: bot.fitness, reverse = True)
    
    for i in range(n_gen):
        if i%100 == 0:
            make_game(population[0])
        print("generation "+str(i))
        population = step_ga(population)   
        population = sorted(population, key=lambda bot: bot.fitness, reverse = True)

In [ ]:
ga(nb_gen, mutation_rate)

Starting generating
[[0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Vous jouez -
['0', '0', '0', '0', '0', '0', '0', '0', '0']
['0', '0', '0', '0', '0', '0', '0', '0', '0']
['0', '0', '0', '0', '0', '0', '0', '0', '0']
['0', '0', '0', '0', '0', '0', '0', '0', '0']
['0', '0', '0', '0', '0', '0', '0', '0', '0']
['0', '0', '0', '0', '0', '0', '0', '0', '0']
['0', '0', '0', '0', '0', '0', '0', '0', '0']
['0', '0', '0', '0', '0', '0', '0', '0', '0']
choisissez un move1
['0', '0', '0', '0', '0', '0', '0', '0', '0']
['0', '-', '0', '0', '0', '*', '0', '0', '0']
['0', '0', '0', '0', '0', '0', '0', '0', '0']
['0', '0', '0', '0', '0', '0', '0', '0', '0']
['0', '0', '0', '0', '0', '0', '0', '0', '0']
['0', '0', '0', '0', '0', '0', '0', '0', '0']
['0', '0', '0', '0', '0', '0', '0', '0', '0']
['0', '0', '0', 

[63, 58, 52, 51, 49, 49, 48, 37, 36, 35, 35, 34, 34, 34, 33, 30, 30, 30, 29, 29, 28, 28, 28, 25, 25, 24, 23, 21, 20, 20, 20, 19, 18, 17, 17, 17, 16, 16, 15, 15, 15, 15, 14, 14, 13, 12, 12, 11, 11, 10, 9, 9, 9, 9, 9, 9, 8, 7, 7, 6, 6, 5, 5, 5, 5, 4, 4, 3, 2, 2, 2, 1, 0, -2, -3, -3, -4, -8, -8, -9, -9, -10, -10, -11, -11, -12, -15, -16, -18, -18, -19, -21, -23, -23, -28, -33, -37, -37, -45, -48]
generation 16
[74, 53, 52, 49, 49, 45, 41, 40, 39, 38, 36, 34, 33, 32, 32, 31, 28, 27, 27, 27, 25, 24, 23, 22, 21, 21, 21, 20, 20, 20, 18, 18, 18, 17, 16, 15, 14, 14, 14, 13, 13, 12, 11, 11, 11, 11, 11, 10, 10, 10, 10, 9, 8, 8, 7, 7, 6, 6, 5, 4, 2, 2, 1, 1, 0, -1, -1, -2, -2, -4, -5, -6, -7, -7, -7, -7, -9, -9, -10, -10, -11, -12, -13, -14, -15, -17, -17, -21, -22, -25, -27, -30, -32, -33, -34, -40, -42, -46, -70, -74]
generation 17
[48, 48, 46, 36, 34, 33, 31, 30, 30, 30, 30, 29, 28, 27, 27, 26, 26, 26, 25, 23, 23, 23, 22, 22, 22, 21, 20, 18, 18, 18, 17, 16, 15, 15, 14, 14, 13, 13, 13, 12, 12, 1

[84, 62, 57, 51, 49, 47, 41, 37, 36, 32, 31, 31, 31, 28, 27, 26, 26, 26, 26, 25, 25, 25, 24, 23, 22, 21, 20, 18, 18, 17, 16, 16, 16, 15, 14, 13, 12, 12, 11, 11, 9, 8, 8, 8, 7, 7, 6, 6, 6, 4, 4, 4, 3, 3, 2, 2, 1, 0, 0, 0, -2, -2, -2, -3, -3, -4, -4, -4, -4, -5, -6, -6, -8, -8, -8, -9, -10, -10, -11, -12, -12, -13, -17, -17, -18, -19, -19, -20, -20, -25, -26, -27, -31, -32, -32, -36, -38, -45, -51, -55]
generation 36
[53, 49, 42, 41, 40, 39, 37, 36, 34, 33, 33, 32, 32, 31, 29, 27, 25, 25, 25, 24, 23, 22, 22, 22, 21, 20, 20, 20, 20, 19, 19, 18, 17, 17, 16, 15, 15, 14, 14, 13, 13, 13, 12, 12, 12, 11, 10, 9, 9, 8, 8, 8, 8, 7, 6, 6, 6, 6, 6, 5, 5, 4, 4, 4, 3, 3, 2, 2, 2, 1, 0, -1, -1, -5, -5, -6, -6, -6, -7, -8, -8, -9, -9, -10, -11, -13, -14, -15, -15, -15, -16, -17, -17, -18, -18, -23, -24, -26, -39, -41]
generation 37
[62, 59, 57, 53, 50, 49, 48, 45, 42, 41, 40, 39, 39, 38, 36, 35, 34, 31, 30, 30, 28, 27, 26, 22, 22, 22, 22, 22, 21, 21, 21, 21, 21, 21, 20, 19, 19, 18, 18, 17, 16, 16, 14, 

[121, 67, 59, 56, 44, 44, 41, 32, 27, 24, 23, 22, 22, 20, 20, 18, 15, 12, 11, 9, 8, 8, 3, 3, 3, 3, 2, 1, 1, 0, -3, -3, -9, -9, -10, -10, -10, -10, -12, -14, -14, -14, -14, -14, -16, -16, -17, -18, -18, -22, -22, -22, -22, -23, -24, -24, -24, -24, -26, -26, -26, -27, -27, -29, -31, -32, -33, -33, -34, -35, -36, -37, -39, -39, -40, -41, -42, -44, -44, -44, -45, -45, -46, -46, -47, -48, -48, -49, -50, -50, -52, -52, -57, -64, -65, -72, -75, -79, -81, -89]
generation 56
[71, 55, 51, 44, 37, 35, 31, 30, 23, 22, 18, 13, 12, 5, 5, 2, 1, 0, 0, -1, -2, -2, -2, -3, -3, -4, -4, -6, -7, -7, -8, -12, -13, -14, -14, -15, -16, -16, -16, -17, -17, -18, -19, -20, -20, -20, -21, -21, -22, -22, -23, -24, -26, -27, -29, -29, -29, -31, -31, -31, -32, -32, -33, -34, -35, -36, -36, -37, -38, -39, -39, -40, -40, -41, -41, -42, -42, -43, -45, -47, -48, -49, -49, -50, -53, -54, -55, -56, -56, -59, -60, -60, -63, -63, -65, -69, -90, -93, -101, -107]
generation 57
[85, 85, 47, 39, 35, 31, 31, 30, 30, 24, 22, 20, 

[68, 56, 36, 33, 33, 33, 32, 30, 28, 24, 23, 21, 17, 17, 14, 14, 12, 10, 8, 7, 6, 5, 4, 3, -3, -4, -5, -8, -9, -10, -12, -12, -15, -15, -15, -15, -18, -19, -19, -21, -22, -22, -23, -25, -26, -27, -28, -30, -30, -30, -31, -31, -32, -32, -33, -33, -34, -34, -35, -36, -36, -38, -39, -39, -40, -42, -43, -43, -44, -46, -48, -48, -49, -50, -53, -57, -57, -61, -61, -63, -63, -69, -70, -70, -70, -73, -74, -75, -76, -77, -80, -83, -83, -84, -84, -84, -86, -88, -92, -100]
generation 73
[47, 43, 42, 41, 37, 36, 27, 20, 20, 19, 19, 18, 15, 6, 6, 3, 2, 1, 0, -1, -3, -3, -5, -8, -8, -9, -9, -10, -10, -11, -13, -17, -18, -19, -20, -21, -21, -22, -22, -22, -22, -24, -24, -25, -26, -28, -28, -28, -28, -29, -29, -30, -31, -31, -31, -32, -32, -34, -35, -35, -35, -36, -36, -36, -36, -39, -39, -40, -41, -43, -44, -44, -46, -47, -50, -51, -51, -52, -52, -53, -53, -56, -58, -60, -61, -61, -62, -71, -72, -74, -75, -81, -82, -84, -88, -90, -94, -96, -98, -104]
generation 74
[38, 25, 20, 20, 14, 12, 12, 11, 7, 

...

[7, 5, 4, 4, 3, 2, 1, -1, -2, -4, -4, -7, -7, -8, -8, -9, -9, -10, -11, -11, -12, -13, -15, -17, -17, -21, -21, -23, -24, -24, -25, -27, -28, -28, -28, -29, -30, -30, -30, -31, -31, -31, -32, -34, -35, -36, -37, -37, -37, -38, -39, -40, -40, -41, -41, -42, -43, -45, -47, -47, -48, -48, -49, -49, -50, -52, -53, -54, -55, -56, -58, -58, -60, -61, -61, -62, -63, -64, -64, -65, -66, -66, -66, -67, -67, -67, -68, -69, -71, -72, -72, -72, -74, -75, -79, -80, -85, -85, -85, -88]
generation 221
[28, 22, 17, 13, 13, 11, 10, 9, 6, 5, 3, 3, 0, -2, -7, -7, -8, -8, -9, -10, -11, -14, -17, -18, -21, -22, -22, -22, -25, -26, -26, -27, -29, -29, -29, -30, -31, -31, -32, -32, -33, -33, -33, -34, -34, -37, -37, -37, -37, -38, -39, -41, -43, -43, -44, -44, -45, -45, -46, -47, -47, -47, -50, -51, -52, -52, -53, -53, -54, -55, -56, -58, -60, -61, -61, -61, -62, -63, -64, -64, -64, -65, -65, -66, -67, -68, -70, -71, -72, -72, -73, -74, -74, -74, -76, -77, -79, -85, -87, -91]
generation 222
[16, 15, 13, 13, 

[29, 15, 13, 12, 3, 2, 2, 0, -2, -3, -6, -8, -11, -11, -11, -12, -12, -13, -13, -14, -14, -17, -18, -20, -22, -22, -23, -24, -24, -26, -26, -26, -26, -28, -29, -30, -30, -31, -31, -32, -33, -33, -33, -34, -34, -34, -35, -35, -35, -35, -36, -40, -40, -41, -42, -43, -44, -44, -44, -45, -46, -48, -49, -50, -51, -51, -51, -51, -52, -53, -55, -59, -59, -60, -60, -61, -63, -64, -64, -65, -66, -66, -67, -67, -68, -68, -69, -70, -71, -72, -72, -73, -75, -76, -76, -76, -81, -84, -88, -97]
generation 238
[15, 13, 8, 7, 4, 2, -2, -3, -3, -4, -5, -7, -11, -11, -11, -13, -13, -13, -13, -14, -14, -16, -18, -18, -19, -19, -21, -21, -22, -23, -23, -25, -25, -25, -26, -26, -28, -28, -29, -29, -30, -30, -33, -34, -35, -37, -37, -37, -37, -38, -38, -38, -39, -39, -40, -40, -42, -43, -43, -44, -45, -46, -47, -47, -48, -52, -53, -55, -55, -56, -56, -56, -57, -58, -58, -59, -59, -60, -61, -62, -63, -63, -68, -69, -69, -72, -72, -72, -73, -73, -73, -77, -77, -77, -79, -79, -80, -83, -86, -97]
generation 239


[16, 14, 8, 7, 4, -1, -1, -2, -2, -2, -4, -4, -5, -8, -8, -9, -9, -10, -11, -11, -13, -14, -17, -17, -18, -20, -20, -20, -21, -24, -24, -24, -25, -25, -25, -26, -27, -29, -29, -30, -34, -34, -36, -37, -38, -38, -39, -39, -39, -40, -40, -41, -41, -41, -42, -42, -43, -44, -45, -45, -45, -46, -47, -47, -50, -50, -50, -52, -52, -53, -53, -55, -56, -58, -59, -61, -61, -62, -63, -64, -64, -66, -67, -68, -69, -70, -71, -72, -72, -73, -74, -75, -76, -78, -78, -80, -80, -83, -86, -86]
generation 255
[15, 14, 13, 10, 8, 7, 2, 0, -1, -4, -5, -5, -10, -11, -11, -13, -14, -17, -17, -18, -18, -18, -18, -18, -20, -20, -22, -23, -24, -25, -26, -26, -27, -27, -28, -28, -31, -32, -33, -33, -33, -33, -34, -34, -34, -34, -35, -35, -36, -36, -37, -37, -38, -38, -38, -39, -40, -40, -40, -41, -42, -43, -46, -47, -48, -49, -52, -55, -56, -57, -57, -58, -58, -59, -60, -61, -61, -61, -64, -64, -65, -65, -65, -67, -68, -69, -69, -70, -72, -73, -74, -74, -75, -77, -81, -82, -84, -87, -87, -89]
generation 256
[10,

[25, 23, 9, 5, 1, 0, 0, 0, -1, -2, -2, -2, -6, -6, -6, -7, -12, -12, -13, -13, -14, -16, -17, -17, -18, -18, -18, -20, -22, -24, -24, -25, -26, -28, -28, -28, -29, -29, -31, -31, -32, -33, -33, -33, -35, -35, -36, -37, -37, -37, -38, -38, -39, -40, -42, -44, -46, -47, -48, -49, -50, -51, -52, -54, -54, -55, -55, -55, -56, -60, -60, -61, -61, -62, -62, -62, -64, -64, -65, -65, -66, -66, -66, -66, -68, -69, -69, -70, -71, -75, -75, -76, -76, -80, -81, -81, -82, -84, -85, -88]
generation 272
[21, 15, 9, 9, 5, 4, 1, 1, -1, -7, -8, -9, -10, -11, -11, -11, -13, -13, -14, -16, -16, -19, -19, -19, -20, -20, -21, -22, -23, -23, -23, -23, -24, -24, -24, -27, -28, -29, -31, -31, -33, -33, -34, -34, -35, -38, -38, -40, -40, -44, -44, -44, -45, -45, -46, -46, -46, -47, -49, -50, -50, -51, -51, -51, -52, -54, -55, -55, -56, -57, -58, -58, -59, -60, -60, -61, -62, -62, -63, -63, -63, -64, -64, -65, -66, -67, -69, -70, -71, -72, -77, -78, -79, -81, -82, -83, -84, -91, -96, -96]
generation 273
[30, 23,

[21, 15, 13, 10, 10, 6, 6, 5, 2, 1, 1, 0, -2, -3, -3, -4, -4, -5, -5, -6, -7, -8, -9, -10, -14, -16, -18, -18, -21, -23, -25, -25, -28, -30, -31, -31, -31, -31, -32, -33, -34, -34, -36, -36, -36, -37, -37, -37, -40, -41, -42, -43, -43, -45, -46, -49, -49, -49, -49, -49, -50, -54, -54, -54, -54, -54, -54, -58, -58, -59, -59, -59, -62, -63, -63, -63, -66, -66, -67, -67, -67, -67, -68, -68, -68, -70, -71, -73, -73, -74, -75, -75, -75, -75, -78, -78, -79, -82, -82, -91]
generation 289
[28, 17, 14, 8, 8, 2, 2, 0, -2, -3, -5, -9, -10, -12, -12, -12, -14, -14, -17, -17, -18, -19, -19, -19, -20, -20, -21, -22, -23, -23, -23, -24, -24, -25, -26, -26, -26, -27, -27, -28, -28, -29, -29, -29, -30, -30, -30, -33, -34, -35, -36, -40, -41, -45, -45, -46, -46, -47, -48, -49, -50, -51, -52, -52, -54, -54, -56, -56, -57, -58, -58, -58, -58, -59, -59, -59, -60, -61, -62, -62, -63, -63, -64, -64, -65, -67, -68, -68, -68, -70, -71, -71, -72, -73, -73, -78, -81, -84, -91, -97]
generation 290
[22, 14, 14, 13

NameError: name 'i' is not defined